Here are some indicative types of analysis you can perform. Please note that this is not an exhaustive list, you may add more. 	Come up with appropriate results  for the followin- \
o	Is there any relation between contract duration, charges and the customer chur \n
o	Is the dataset balanc \e
o	Identify the relationship between the services offered and the churn r \a.
o	Demographic analysis of the customers.


In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

pd.set_option('display.max_columns', None)

## Data Loading

In [13]:
data_dir_path = os.path.abspath('dev/Deloitte_capstone_project/TelecomData')
data_dir_path

'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\TelecomData'

In [314]:
datafile_names = os.listdir(data_dir_path)
datafile_paths = [os.path.join(data_dir_path, datafile) for datafile in datafile_names]

datafile_paths

['C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\TelecomData\\Customer_account_info.xlsx',
 'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\TelecomData\\Customer_Churn.csv',
 'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\TelecomData\\Customer_demographics.xlsx',
 'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\TelecomData\\Customer_services.xlsx']

In [42]:
cust_acc_df = pd.read_excel(datafile_paths[0])
cust_churn_df = pd.read_csv(datafile_paths[1])
cust_demo_df = pd.read_excel(datafile_paths[2])
cust_serv_df = pd.read_excel(datafile_paths[3])

In [50]:
cust_acc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34413 entries, 0 to 34412
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        34413 non-null  object 
 1   tenure            34397 non-null  float64
 2   Contract          34406 non-null  object 
 3   PaperlessBilling  34380 non-null  object 
 4   PaymentMethod     34364 non-null  object 
 5   MonthlyCharges    34400 non-null  float64
 6   TotalCharges      34403 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.8+ MB


In [52]:
cust_churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34413 entries, 0 to 34412
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  34413 non-null  object
 1   Churn       34413 non-null  object
dtypes: object(2)
memory usage: 537.8+ KB


In [54]:
cust_demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34413 entries, 0 to 34412
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customerID     34413 non-null  object 
 1   gender         34408 non-null  object 
 2   SeniorCitizen  34371 non-null  float64
 3   Partner        34413 non-null  object 
 4   Dependents     34413 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.3+ MB


In [56]:
cust_serv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34413 entries, 0 to 34412
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        34413 non-null  object
 1   PhoneService      34401 non-null  object
 2   MultipleLines     34413 non-null  object
 3   InternetService   34413 non-null  object
 4   OnlineSecurity    34376 non-null  object
 5   OnlineBackup      34413 non-null  object
 6   DeviceProtection  34413 non-null  object
 7   TechSupport       34378 non-null  object
 8   StreamingTV       34413 non-null  object
 9   StreamingMovies   34413 non-null  object
dtypes: object(10)
memory usage: 2.6+ MB


## Data cleaning

**Extracting integers from customerID values and replacing them**

In [108]:
reg_exp_cid = r'\d+'
extracted_cids = cust_churn_df['customerID'].apply(lambda x: int(re.search(reg_exp_cid, x).group())).tolist()

cust_churn_df['customerID'] = extracted_cids
cust_acc_df['customerID'] = extracted_cids
cust_demo_df['customerID'] = extracted_cids
cust_serv_df['customerID'] = extracted_cids

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34413 entries, 0 to 34412
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  34413 non-null  int64 
 1   Churn       34413 non-null  object
dtypes: int64(1), object(1)
memory usage: 537.8+ KB


In [159]:
cust_acc_df = cust_acc_df.rename(columns={'tenure':'Tenure'})
cust_demo_df = cust_demo_df.rename(columns={'gender': 'Gender'})

### Exploring and Cleaning accounts dataframe

In [163]:
cust_acc_df['Contract'].value_counts(dropna=False)

Contract
Month-to-month    19693
Two year           9823
One year           4890
NaN                   7
Name: count, dtype: int64

In [165]:
cust_acc_df['PaperlessBilling'].value_counts(dropna=False)

PaperlessBilling
Yes    21982
No     12398
NaN       33
Name: count, dtype: int64

In [167]:
cust_acc_df['PaymentMethod'].value_counts(dropna=False)

PaymentMethod
Mailed check                 10643
Electronic check              9566
Bank transfer (automatic)     8656
Credit card (automatic)       5499
NaN                             49
Name: count, dtype: int64

In [245]:
cust_acc_df.loc[~cust_acc_df['TotalCharges'].apply(lambda x: x.isdigit() if isinstance(x, str) else True), 'TotalCharges']

2020      
4431      
4518      
14108     
14358     
15320     
15539     
23409     
23684     
24350     
28679     
30310     
Name: TotalCharges, dtype: object

TotalCharges column is of type object and have a few rows where the value is empty string. We can replace these with null values

In [263]:
cust_acc_df.loc[~cust_acc_df['TotalCharges'].apply(lambda x: x.isdigit() if isinstance(x, str) else True), 'TotalCharges'] = np.nan
cust_acc_df['TotalCharges'] = cust_acc_df['TotalCharges'].astype('float')

In [265]:
cust_acc_df.describe()

,customerID,Tenure,MonthlyCharges,TotalCharges
count,34413.000000,34397.000000,34400.000000,34391.000000
mean,17207.000000,32.912812,65.049760,2493.521648
std,9934.321743,21.396691,27.496134,2417.484468
min,1.000000,0.000000,18.250000,18.800000
25%,8604.000000,16.000000,44.289520,428.700000
50%,17207.000000,32.000000,64.883350,1533.800000
75%,25810.000000,49.000000,84.930166,4284.650000
max,34413.000000,72.000000,118.750000,8684.800000


Accounts table values look clean, except the null values

### Exploring and cleaning demographics dataframe

In [283]:
cust_demo_df.columns.tolist()

['customerID', 'Gender', 'SeniorCitizen', 'Partner', 'Dependents']

In [281]:
cust_demo_df['Gender'].value_counts(dropna=False)

Gender
Male      17490
Female    16918
NaN           5
Name: count, dtype: int64

In [285]:
cust_demo_df['SeniorCitizen'].value_counts(dropna=False)

SeniorCitizen
0.0    28210
1.0     6161
NaN       42
Name: count, dtype: int64

In [287]:
cust_demo_df['Partner'].value_counts(dropna=False)

Partner
No     17901
Yes    16512
Name: count, dtype: int64

In [289]:
cust_demo_df['Dependents'].value_counts(dropna=False)

Dependents
No     26995
Yes     7418
Name: count, dtype: int64

### Exploring and cleaning servies table

In [294]:
cust_serv_df.columns.tolist()

['customerID',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies']

In [298]:
for col in cust_serv_df.columns:
    if col == 'customerID':
        continue
    print(cust_serv_df[col].value_counts(dropna=False))
    print("++++++++++++++++++++++++++++++++")

PhoneService
Yes    33520
No       881
NaN       12
Name: count, dtype: int64
++++++++++++++++++++++++++++++++
MultipleLines
No                  16150
Yes                 15931
No phone service     2332
Name: count, dtype: int64
++++++++++++++++++++++++++++++++
InternetService
DSL            15006
Fiber optic    14181
No              5226
Name: count, dtype: int64
++++++++++++++++++++++++++++++++
OnlineSecurity
No                     16890
Yes                    12216
No internet service     5270
NaN                       37
Name: count, dtype: int64
++++++++++++++++++++++++++++++++
OnlineBackup
Yes                    15080
No                     14064
No internet service     5269
Name: count, dtype: int64
++++++++++++++++++++++++++++++++
DeviceProtection
Yes                    15037
No                     14047
No internet service     5329
Name: count, dtype: int64
++++++++++++++++++++++++++++++++
TechSupport
No                     16860
Yes                    12341
No internet servic

## Writing dataframe

In [304]:
output_folder_path = os.path.abspath("dev/Deloitte_capstone_project/Output_telecomData")

In [316]:
inner_output_folder_name = "WithNaNs"
inner_output_folder_path = os.path.join(output_folder_path, inner_output_folder_name)

if not os.path.exists(inner_output_folder_path):
    os.makedirs(inner_output_folder_path)

output_file_paths = [os.path.join(inner_output_folder_path, output_filename) for output_filename in datafile_names]
print(output_file_paths)

['C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\Output_telecomData\\WithNaNs\\Customer_account_info.xlsx', 'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\Output_telecomData\\WithNaNs\\Customer_Churn.csv', 'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\Output_telecomData\\WithNaNs\\Customer_demographics.xlsx', 'C:\\Users\\shreyanlnu\\dev\\Deloitte_capstone_project\\Output_telecomData\\WithNaNs\\Customer_services.xlsx']


In [318]:
cust_acc_df.to_csv(output_file_paths[0])
cust_churn_df.to_csv(output_file_paths[1])
cust_demo_df.to_csv(output_file_paths[2])
cust_serv_df.to_csv(output_file_paths[3])